In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
plt.style.use('seaborn')
plt.rcParams.update({'figure.figsize': (16,8)})
sns.set_context("talk")

In [ ]:
import os
os.listdir('data')

In [ ]:
INPATH = "data/"
INFILE1 = "cnmc_goa_gna.xlsx"

In [ ]:
OUTPATH = 'data/out/'
OUTFILE = 'cons_gna95_naive.xlsx'

In [ ]:
data = pd.read_excel(INPATH+INFILE1, parse_dates=['Date'], index_col='Date')
data.index.freq = 'MS'
data.info()

In [ ]:
cols = data.select_dtypes(include='number').columns.tolist()

In [ ]:
re_cons = re.compile(r'^cons_')
re_spot = re.compile(r'^spot_')
re_price = re.compile(r'^price_')

In [ ]:
ax = data.filter(regex=re_cons).plot(title='Consumption')
ax.set_ylabel('(t)')
plt.show()

In [ ]:
data.filter(regex=re_spot).plot(title='spot prices')
ax.set_ylabel('($/t)')
plt.show()

In [ ]:
data.filter(regex=re_price).plot(title='consumer prices')
ax.set_ylabel('(€/l)')
plt.show()

# Basic feature engineering

In [ ]:
cols_pct_change = []
for col in cols:
    data[f'{col}_pct_change1'] = data[col].pct_change()
    data[f'{col}_diff1'] = data[col].diff()
    data[f'{col}_diff1_sdiff12'] =  data[f'{col}_diff1'].diff(12)
    
    data[f'{col}_mave3'] = data[col].rolling(3).mean()
    data[f'{col}_vs_mave3'] = data[col]- data[f'{col}_mave3']
    
    data[f'{col}_pct_change1_lag1'] = data[f'{col}_pct_change1'].shift(1)
    data[f'{col}_diff1_lag1'] = data[f'{col}_diff1'].shift(1)
    
    
data['yr'] = data.index.year
data['qtr'] = data.index.quarter
data['mon'] = data.index.month
data['week'] = data.index.week

# Seasonal and Trend Decomposition

In [ ]:
seas_period = 12
seas_models = {}
for col in cols:
    # Fit decomposition model with train data
    seas_add = seasonal_decompose(data.loc[data['split'] == 'train', col], period=seas_period, model='additive')
    seas_models[col] = seas_add
    seas_comp = seas_add.seasonal.copy()
    seas_comp.index = seas_comp.index.month
    # Index by seasonal period
    seas_dict = seas_comp[:seas_period].to_dict()
    data[f'{col}_seas_comp'] = data['mon'].replace(seas_dict)
    data[f'{col}_seas_adj'] = data[col] -  data[f'{col}_seas_comp']
    # Compute other features
    data[f'{col}_seas_adj_diff1'] =  data[col].diff()
    data[f'{col}_seas_adj_pct_change1'] = data[f'{col}_seas_adj'].pct_change()
    data[f'{col}_seas_adj_pct_change1_lag1'] = data[f'{col}_seas_adj_pct_change1'].shift(1)
    
    data[f'{col}_seas_adj_mave{seas_period}'] = data[col].rolling(seas_period).mean()
    data[f'{col}_seas_adj_vs_mave{seas_period}'] = data[col] - data[f'{col}_seas_adj_mave{seas_period}']

In [ ]:
data.isna().sum(axis=0)

## Autorregressive model features and targets

In [ ]:
curr_features = [f'cons_GNA95_pct_change1', 'price_GNA95_pct_change1']
ar_features = []
for col in ['cons_GNA95_pct_change1']:
    for tau in range(1, 3+1):
        data[f'{col}_lag{tau}'] = data[f'{col}'].shift(tau)
        ar_features.append(f'{col}_lag{tau}')

In [ ]:
extra_features = ['cons_GNA95_pct_change1_zstd']

data['cons_GNA95_pct_change1_mave12'] = data['cons_GNA95_pct_change1'].rolling(12).mean()
data['cons_GNA95_pct_change1_mstd12'] = data['cons_GNA95_pct_change1'].rolling(12).std()
data['cons_GNA95_pct_change1_zstd'] =  (data['cons_GNA95_pct_change1'] - data['cons_GNA95_pct_change1_mave12'])/data['cons_GNA95_pct_change1_mstd12']

In [ ]:
data['x_mon_seas_cos'] = np.cos(data['mon']*2*np.pi/12)
data['x_mon_seas_sin'] = np.sin(data['mon']*2*np.pi/12)
seas_features = ['x_mon_seas_cos', 'x_mon_seas_sin']

In [ ]:
data['cons_GNA95_pct_change1_t1'] = data['cons_GNA95_pct_change1'].shift(-1)

In [ ]:
data.plot.scatter(x='x_mon_seas_cos', y='x_mon_seas_sin',figsize=(4,4))
plt.show()

In [ ]:
data = data[13:].copy()

In [ ]:
data.isna().sum(axis=0).sum()

In [ ]:
data_train = data.loc[data['split']=='train'].copy()
data_test = data.loc[data['split']=='test'].copy()

# Benchmark: Naive

In [ ]:
p_gna95_train = data_train[['cons_GNA95']].copy()

p_gna95_train['p_cons_GNA95'] = data_train['cons_GNA95'].shift(1)
p_gna95_train['model'] = 'naive'
p_gna95_train['split'] = 'train'
p_gna95_train.index = p_gna95_train.index.date

In [ ]:
p_gna95_test = data_test[['cons_GNA95']].copy()

p_gna95_test['p_cons_GNA95'] = data_test['cons_GNA95'].shift(1)
p_gna95_test['p_cons_GNA95'].iloc[0] = data_train['cons_GNA95'].iloc[-1]
p_gna95_test['model'] = 'naive'
p_gna95_test['split'] = 'test'
p_gna95_test.index = p_gna95_test.index.date

In [ ]:
select_cols = ['model', 'split', 'cons_GNA95', 'p_cons_GNA95']

pd.concat([p_gna95_train, p_gna95_test], axis=0)[select_cols].to_excel(OUTPATH + OUTFILE, index_label='Date')